In [10]:
pip install wrds



[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
!pip install sqlalchemy




[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [25]:
pip install pandas_datareader


  Using cached pandas_datareader-0.10.0-py3-none-any.whl.metadata (2.9 kB)
Using cached pandas_datareader-0.10.0-py3-none-any.whl (109 kB)

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
# Folder: scripts/ingest_compustat.py
import wrds
import pandas as pd
from sqlalchemy import create_engine

In [15]:
# WRDS login and parameters
start_date = '2005-01-01'
end_date = '2023-12-31'
db_path = "/Users/niveditavats/Documents/Projects_CC/equity_analytics_pipelines/database/investment_data.sqlite"

In [20]:
# start_date = "2005-01-01"
# end_date = "2023-12-31"
# db_path = "/Users/niveditavats/Documents/Projects_CC/equity_analytics_pipelines/database/investment_data.sqlite"


In [16]:
# Connect to WRDS and SQLite
conn = wrds.Connection()
engine = create_engine(f"sqlite:///{db_path}")

Loading library list...
Done


In [8]:
conn.list_table_columns(library='comp', table='fundq')


AttributeError: 'Connection' object has no attribute 'list_table_columns'

In [19]:
# Query Compustat fundamentals (quarterly) — using confirmed columns only
query = f"""
    SELECT gvkey, datadate, atq AS at, ltq AS lt, saleq AS sale, cogsq AS cogs, xintq AS xint, xsgaq AS xsga, oancfy AS oancf
    FROM comp.fundq
    WHERE indfmt = 'INDL'
      AND datafmt = 'STD'
      AND consol = 'C'
      AND popsrc = 'D'
      AND datadate BETWEEN '{start_date}' AND '{end_date}'
"""
data = conn.raw_sql(query)

In [20]:

# Example mapping (replace with CCM linking table in future)
data['symbol'] = 'AAPL'
data['datadate'] = pd.to_datetime(data['datadate'])
data['book_equity'] = data['at'] - data['lt']

In [21]:
# Compute profitability
with pd.option_context('mode.use_inf_as_na', True):
    data['profitability'] = (data['sale'] - data['cogs'] - data['xsga'] - data['xint']) / data['at']


In [22]:
# Write to database
data.to_sql("compustat_quarterly", con=engine, if_exists="replace", index=False)
print("Compustat data loaded from WRDS and stored in database.")



Compustat data loaded from WRDS and stored in database.


In [23]:


# Folder: scripts/ingest_crsp.py
import wrds
import pandas as pd
from sqlalchemy import create_engine

start_date = '2020-01-01'
end_date = '2023-12-31'
db_path = '../database/investment_data.sqlite'

conn = wrds.Connection()
engine = create_engine(f"sqlite:///{db_path}")

query = f"""
    SELECT permno, date, ret
    FROM crsp.dsf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
      AND permno IN (14593)  -- Example: AAPL permno, replace with mapped permnos
"""

prices = conn.raw_sql(query)
prices['symbol'] = 'AAPL'
prices['date'] = pd.to_datetime(prices['date'])
prices = prices[prices['ret'].notnull()]
prices.to_sql("crsp_daily", con=engine, if_exists="replace", index=False)
print("CRSP daily returns pulled from WRDS and stored in database.")



Loading library list...
Done
CRSP daily returns pulled from WRDS and stored in database.


In [26]:
# Folder: scripts/ingest_fama_french.py
import pandas_datareader.data as web
from sqlalchemy import create_engine
import datetime

start = datetime.datetime(2005, 1, 1)
end = datetime.datetime(2023, 12, 31)
db_path = "../database/investment_data.sqlite"

engine = create_engine(f"sqlite:///{db_path}")
df = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench", start, end)[0]
df = df.reset_index().rename(columns={
    'Mkt-RF': 'mktrf', 'SMB': 'smb', 'HML': 'hml', 'RF': 'rf', 'Date': 'date'
})
df['date'] = pd.to_datetime(df['date'])
df.to_sql("fama_french_daily", con=engine, if_exists="replace", index=False)
print("Fama-French daily factors ingested and stored in database.")


/var/folders/_j/hd0_zxdd7ms7hgqjnv9jqw280000gn/T/ipykernel_5541/2330263633.py:11: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench", start, end)[0]


Fama-French daily factors ingested and stored in database.


In [30]:
# Folder: scripts/merge_and_compute_returns.py
import pandas as pd
from sqlalchemy import create_engine

engine = create_engine("sqlite:///../database/investment_data.sqlite")

comp = pd.read_sql("SELECT * FROM compustat_quarterly", con=engine)
comp['datadate'] = pd.to_datetime(comp['datadate'])
crsp = pd.read_sql("SELECT * FROM crsp_daily", con=engine)
crsp['date'] = pd.to_datetime(crsp['date'])
ff = pd.read_sql("SELECT * FROM fama_french_daily", con=engine)

crsp['quarter'] = crsp['date'].dt.to_period('Q')
returns_qtr = crsp.groupby(['symbol', 'quarter'])['ret'].apply(lambda x: (1 + x).prod() - 1).reset_index()
returns_qtr = returns_qtr.rename(columns={'ret': 'qtr_return'})

comp['quarter'] = comp['datadate'].dt.to_period('Q')

merged = pd.merge(comp, returns_qtr, on=['symbol', 'quarter'], how='inner')
merged['quarter'] = merged['quarter'].astype(str)
merged.to_sql("merged_fundamentals_returns", con=engine, if_exists="replace", index=False)
print("Merged data with quarterly returns stored in database.")


Merged data with quarterly returns stored in database.
